In [14]:
import spacy
import pandas as pd
from spacy import displacy
import os
import re

In [3]:
# Cargar el archivo CSV
df = pd.read_csv(r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\limpios\dialogos_procesados.csv")

# Ver las primeras filas para entender la estructura
print(df.head())

               titulo                                            dialogo
0            Clitofon   Clitofon, acusado por Sócrates de haber censu...
1               Fedón   El El relato histórico consiste en la pintura...
2     Timeo de Locres   Timeo, nacido entre los locrios epicefirianos...
3  segundo Alcibíades   No debe orarse ligeramente. Dirigir a los dio...
4            Clitofon   Sócrates Clitofon, hijo de Aristonimo, me han...


In [4]:
def limpiar_texto(texto):
    # Esta expresión regular busca cualquier cosa entre corchetes, incluyendo el espacio que pueda quedar antes
    return re.sub(r'\s*\[\d+\]', '', texto)

In [5]:
# Carga modelo español large
nlp = spacy.load('es_core_news_lg')

In [6]:
# Ejemplo de un diálogo específico
ejemplo_dialogo = df["dialogo"][0]  # Primer diálogo
doc = nlp(ejemplo_dialogo)

In [7]:
# Visualización de dependencias sintácticas
displacy.render(doc, style="dep", jupyter=True)

In [8]:
# Visualizar entidades nombradas
displacy.render(doc, style="ent", jupyter=True)

In [9]:
"""
Funcion procesamiento de texto
Tokenizacion
Lematizacion convertir palabras a su forma base
Eliminacion de stopwords (Quita palabras sin valor semántico)
"""

def procesar_texto(texto):
    doc = nlp(texto)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)

In [10]:
# Aplicamos la función a los diálogos
df["dialogo_procesado"] = df["dialogo"].apply(procesar_texto)

# Mostrar primeros ejemplos
print(df[["titulo", "dialogo_procesado"]].head())

               titulo                                  dialogo_procesado
0            Clitofon    clitofon acusado sócrates censurar conversac...
1               Fedón    relato histórico consistir pintura sensible ...
2     Timeo de Locres    timeo nacido locrio epicefiriano grecia cuyo...
3  segundo Alcibíades    orar él ligeramente dirigir dios súplica ped...
4            Clitofon    sócrates clitofon hijo aristonimo instante c...


In [11]:
"""
Socrates usaba preguntas para guiar el pensamiento.
Vamos a extraer todas las preguntas de los dialogos.
"""
def extraer_preguntas(texto):
    doc = nlp(texto)
    preguntas = [sent.text for sent in doc.sents if "?" in sent.text]
    return preguntas

In [15]:
df["preguntas_socraticas"] = df["dialogo"].apply(extraer_preguntas)

# Mostrar las preguntas de los diálogos
for index, row in df.iterrows():
    print(f"{row['titulo']} - Preguntas de Sócrates:")
    for pregunta in row["preguntas_socraticas"]:
        pregunta_limpia = limpiar_texto(pregunta)
        print(f"{pregunta_limpia}")

Clitofon - Preguntas de Sócrates:
Fedón - Preguntas de Sócrates:
¿No debe el filósofo desear morir?
¿Tiene el derecho de decidir, según su voluntad, la muerte que tarda demasiado en venir, y no esperar el plazo del destino?
¿Pero qué son el precio de la vida y el terror de la muerte para el que no da al cuerpo ningún valor?
¿Pero de dónde procede la certidumbre del filósofo de que con la muerte no perece todo él?
Y no teniendo la prueba de que el alma debe sobrevivir al cuerpo, ¿quién le asegura que no sea esto un engaño y una bella ilusión?
¿No es cierto que nuestra alma, al través de la imperfecta igualdad que muestran los objetos sensibles entre sí, tiene la idea de una igualdad perfecta, inteligible e inaccesible a los sentidos?
¿No tiene asimismo la idea del bien, de lo justo, de lo santo y de la esencia de todas las cosas?
Nuestra alma, ¿es una de las cosas que pueden disolverse, o es indisoluble?
¿Es simple o compuesta, material o inmaterial?
En fin, ¿con qué se conforma más; co

In [16]:
"""
Podemos filtrar los dáilogos donde Sócrates es el que habla. detectando su nombre al inicio de las lineas.
"""
def extraer_dialogo_socratico(texto):
    lineas = texto.split("\n")
    dialogo_socratico = [linea for linea in lineas if linea.strip().startswith("Sócrates")]
    return " ".join(dialogo_socratico)

In [17]:
df["dialogo_socratico"] = df["dialogo"].apply(extraer_dialogo_socratico)

# Mostrar diálogos donde habla Sócrates
print(df[["titulo", "dialogo_socratico"]].head(20))

                   titulo                                  dialogo_socratico
0                Clitofon                                                   
1                   Fedón                                                   
2         Timeo de Locres                                                   
3      segundo Alcibíades                                                   
4                Clitofon   Sócrates Clitofon, hijo de Aristonimo, me han...
5         Timeo de Locres                                                   
6                Eutidemo                                                   
7             El banquete                                                   
8   El segundo Alcibíades   Sócrates Alcibíades, ¿vas a orar en este temp...
9                  Filebo                                                   
10                  Fedón                                                   
11                  Fedro                                                   

In [18]:
"""
Usamos Named Entity Recognition para identificar conceptos clave en los diálogos socraticos.
"""
def extraer_entidades(texto):
    doc = nlp(texto)
    entidades = [(ent.text, ent.label_) for ent in doc.ents]
    return entidades

In [19]:
df["entidades_socraticas"] = df["dialogo_socratico"].apply(extraer_entidades)

# Mostrar entidades reconocidas
print(df[["titulo", "entidades_socraticas"]].head())

               titulo                               entidades_socraticas
0            Clitofon                                                 []
1               Fedón                                                 []
2     Timeo de Locres                                                 []
3  segundo Alcibíades                                                 []
4            Clitofon  [(Sócrates Clitofon, PER), (Aristonimo, PER), ...


In [20]:
# Definir el directorio de salida
output_directory = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\procesed"

# Guardar los DataFrames procesados
df.to_csv(os.path.join(output_directory, "dialogos_socraticos_procesados.csv"), index=False)

